<a href="https://colab.research.google.com/github/kattens/Protein-Interaction-with-LLMs/blob/main/First_part_Pre_training_bert_model_on_new_tokens.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# pretraining the ProtBERT
in this part of the code we want to pretrain a bert model that is already pretrained on single protein inputs. we want the model to learn to understand a pair of protein as part of our input that have the tokens to seperate each protein sequence in the pair.

In [ ]:
import pandas as pd
import numpy as np
import torch

print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CUDA not available")

# Check if CUDA is available, else use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

True
Tesla T4
Using device: cuda


In [ ]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pairs= pd.read_csv('/content/drive/MyDrive/pairs.csv')

In [ ]:
# Filter out rows where 'sequence_A' contains '?'
df = pairs[~pairs['masked_sequence_B'].str.contains('\?', na=False)]


In [ ]:
df.columns

Index(['pair_id', 'Protein Name A', 'Protein Name B', 'masked_sequence_A',
       'masked_sequence_B', 'coords_A', 'coords_B', 'Embeddings_A',
       'Sequence_A', 'Embeddings_B', 'Sequence_B', 'tokenized_sequence_A',
       'tokenized_sequence_B', 'tokenized_masked_sequence_A',
       'tokenized_masked_sequence_B', 'sum_tokenized_sequence_A',
       'sum_tokenized_sequence_B'],
      dtype='object')

In [ ]:
pairs_df = df

# Truncate each specified column to a maximum length of 500 characters
columns = ['masked_sequence_A', 'masked_sequence_B', 'Sequence_A', 'Sequence_B']
for col in columns:
    pairs_df[col] = pairs_df[col].apply(lambda x: x[:500] if len(x) > 500 else x)

# Find the longest string by length just to check if the code above works
pairs_df['Length'] = pairs_df['masked_sequence_A'].apply(len)
longest_string = pairs_df.loc[pairs_df['Length'].idxmax(), 'masked_sequence_A']
print(len(longest_string))


500


<ipython-input-9-40379decdd50>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pairs_df[col] = pairs_df[col].apply(lambda x: x[:500] if len(x) > 500 else x)
<ipython-input-9-40379decdd50>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pairs_df[col] = pairs_df[col].apply(lambda x: x[:500] if len(x) > 500 else x)
<ipython-input-9-40379decdd50>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cavea

In [ ]:
pairs_df

,pair_id,Protein Name A,Protein Name B,masked_sequence_A,masked_sequence_B,coords_A,coords_B,Embeddings_A,Sequence_A,Embeddings_B,Sequence_B,tokenized_sequence_A,tokenized_sequence_B,tokenized_masked_sequence_A,tokenized_masked_sequence_B,sum_tokenized_sequence_A,sum_tokenized_sequence_B,Length
0,1H0J,1H0J_B,1H0J_C,---------------------------A------------------...,----------------------------T-----------------...,"[(15.18, 27.908, -3.171), (13.094, 28.367, -0....","[(31.71, 11.761, 52.412), (31.514, 9.193, 49.6...","tensor([ 0.1932, -0.0438, 0.7770, 0.2603, -0...",LKCNKLVPLFYKTCPAGKNLCYKMFMVATPKVPVKRGCIDVCPKSS...,"tensor([-1.9606, -0.0789, -1.0516, 1.0233, -1...",LKCNKLVPLFYKTCPAGKNLCYKMFMVATPKVPVKRGCIDVCPKSS...,"[5, 12, 23, 17, 12, 5, 8, 16, 5, 19, 20, 12, 1...","[5, 12, 23, 17, 12, 5, 8, 16, 5, 19, 20, 12, 1...","[32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 3...","[32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 3...","[37, 44, 55, 49, 44, 37, 40, 48, 37, 51, 52, 4...","[37, 44, 55, 49, 44, 37, 40, 48, 37, 51, 52, 4...",60
1,1H0J,1H0J_B,1H0J_A,---------------------------------V------------...,---------------------------------V------------...,"[(15.18, 27.908, -3.171), (13.094, 28.367, -0....","[(47.873, 32.351, 21.457), (46.719, 29.304, 23...","tensor([ 0.1932, -0.0438, 0.7770, 0.2603, -0...",LKCNKLVPLFYKTCPAGKNLCYKMFMVATPKVPVKRGCIDVCPKSS...,"tensor([ 0.2015, 0.3117, -0.5839, 1.0137, -0...",LKCNKLVPLFYKTCPAGKNLCYKMFMVATPKVPVKRGCIDVCPKSS...,"[5, 12, 23, 17, 12, 5, 8, 16, 5, 19, 20, 12, 1...","[5, 12, 23, 17, 12, 5, 8, 16, 5, 19, 20, 12, 1...","[32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 3...","[32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 3...","[37, 44, 55, 49, 44, 37, 40, 48, 37, 51, 52, 4...","[37, 44, 55, 49, 44, 37, 40, 48, 37, 51, 52, 4...",60
3,1H1I,1H1I_A,1H1I_C,----------------------------------------------...,----------------------------------------------...,"[(65.645, 40.726, 70.958), (67.22, 39.56, 67.6...","[(77.459, 39.1, 34.851), (76.18, 36.721, 37.4)...","tensor([-0.7782, -2.3356, -0.4710, 0.6213, -2...",SSLIVEDAPDHVRPYVIRHYSHARAVTVDTQLYRFYVTGPSSGYAF...,"tensor([-6.7764e-01, 3.7070e-02, 2.5174e-01,...",SSLIVEDAPDHVRPYVIRHYSHARAVTVDTQLYRFYVTGPSSGYAF...,"[10, 10, 5, 11, 8, 9, 14, 6, 16, 14, 22, 8, 13...","[10, 10, 5, 11, 8, 9, 14, 6, 16, 14, 22, 8, 13...","[32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 3...","[32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 3...","[42, 42, 37, 43, 40, 41, 46, 38, 48, 46, 54, 4...","[42, 42, 37, 43, 40, 41, 46, 38, 48, 46, 54, 4...",344
4,1H1I,1H1I_B,1H1I_C,---------D--------------------------------G---...,---------D--------------------------------G---...,"[(67.215, 3.653, 5.427), (68.427, 7.123, 4.339...","[(77.459, 39.1, 34.851), (76.18, 36.721, 37.4)...","tensor([-0.4936, 1.1125, 1.4919, 0.1379, -1...",SSLIVEDAPDHVRPYVIRHYSHARAVTVDTQLYRFYVTGPSSGYAF...,"tensor([-6.7764e-01, 3.7070e-02, 2.5174e-01,...",SSLIVEDAPDHVRPYVIRHYSHARAVTVDTQLYRFYVTGPSSGYAF...,"[10, 10, 5, 11, 8, 9, 14, 6, 16, 14, 22, 8, 13...","[10, 10, 5, 11, 8, 9, 14, 6, 16, 14, 22, 8, 13...","[32, 32, 32, 32, 32, 32, 32, 32, 32, 14, 32, 3...","[32, 32, 32, 32, 32, 32, 32, 32, 32, 14, 32, 3...","[42, 42, 37, 43, 40, 41, 46, 38, 48, 28, 54, 4...","[42, 42, 37, 43, 40, 41, 46, 38, 48, 28, 54, 4...",346
5,1H1I,1H1I_B,1H1I_D,----------------------------------------------...,----------------------------------------------...,"[(67.215, 3.653, 5.427), (68.427, 7.123, 4.339...","[(66.547, 10.408, -23.921), (67.719, 13.907, -...","tensor([-0.4936, 1.1125, 1.4919, 0.1379, -1...",SSLIVEDAPDHVRPYVIRHYSHARAVTVDTQLYRFYVTGPSSGYAF...,"tensor([-0.7012, -1.2548, -0.1944, 1.1893, 2...",SLIVEDAPDHVRPYVIRHYSHARAVTVDTQLYRFYVTGPSSGYAFT...,"[10, 10, 5, 11, 8, 9, 14, 6, 16, 14, 22, 8, 13...","[10, 5, 11, 8, 9, 14, 6, 16, 14, 22, 8, 13, 16...","[32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 3...","[32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 3...","[42, 42, 37, 43, 40, 41, 46, 38, 48, 46, 54, 4...","[42, 37, 43, 40, 41, 46, 38, 48, 46, 54, 40, 4...",346
.

#Adding New Tokens:

We will introduce new tokens, such as ['[ENTITY1]', '[ENTITY2]'], to track relationships between various chains along with their global and local information, enabling the model to recognize these as part of a single entity. These tokens should be incorporated into the ProtBERT tokenizer, followed by training the model's layers with these tokens to facilitate its learning and understanding of their significance.


In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

# Initialize the ProtBERT tokenizer
tokenizer = AutoTokenizer.from_pretrained('Rostlab/prot_bert')

# Define special tokens for entities
special_tokens = ['[ENTITY1]', '[ENTITY2]', '-']

# Add special tokens to the tokenizer
tokenizer.add_special_tokens({'additional_special_tokens': special_tokens})

# Check if the special tokens were added successfully
print(f"Token '[ENTITY1]' has ID: {tokenizer.convert_tokens_to_ids('[ENTITY1]')}")
print(f"Token '[ENTITY2]' has ID: {tokenizer.convert_tokens_to_ids('[ENTITY2]')}")
print(f"Token '-' has ID: {tokenizer.convert_tokens_to_ids('-')}")

# Initialize the ProtBERT model configured for Masked Language Modeling
model = AutoModelForMaskedLM.from_pretrained('Rostlab/prot_bert')
model.resize_token_embeddings(len(tokenizer))  # Adjust the model's embedding size to accommodate new tokens
print('Token embeddings resized to accommodate new tokens.')

# Helper function to convert numerical token IDs back to their textual representation
def ids_to_text(ids):
    return ' '.join(tokenizer.convert_ids_to_tokens(ids))

# Check the updated size of the tokenizer's vocabulary
print(f"Updated vocabulary size: {len(tokenizer)}")

if '[ENTITY1]' in tokenizer.get_vocab() and '[ENTITY2]' in tokenizer.get_vocab() and '-' in tokenizer.get_vocab():
    print("[ENTITY1] and [ENTITY2] and - are in the tokenizer's vocabulary.")
else:
    print("[ENTITY1] and [ENTITY2] and - are NOT in the tokenizer's vocabulary.")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/86.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/361 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/81.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Token '[ENTITY1]' has ID: 30
Token '[ENTITY2]' has ID: 31
Token '-' has ID: 32


pytorch_model.bin:   0%|          | 0.00/1.68G [00:00<?, ?B/s]

Some weights of the model checkpoint at Rostlab/prot_bert were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Token embeddings resized to accommodate new tokens.
Updated vocabulary size: 33
[ENTITY1] and [ENTITY2] and - are in the tokenizer's vocabulary.


In [ ]:
vocab = tokenizer.get_vocab()

In [ ]:
#just a showcase of all the tokens
vocab

{'D': 14,
 'C': 23,
 'B': 27,
 '[ENTITY1]': 30,
 'Q': 18,
 'N': 17,
 '[PAD]': 0,
 'I': 11,
 '[UNK]': 1,
 'M': 21,
 'A': 6,
 'W': 24,
 'E': 9,
 'Z': 28,
 'P': 16,
 'H': 22,
 'F': 19,
 'G': 7,
 'V': 8,
 '[MASK]': 4,
 'T': 15,
 'Y': 20,
 'K': 12,
 '[SEP]': 3,
 'L': 5,
 'S': 10,
 '-': 32,
 'R': 13,
 '[ENTITY2]': 31,
 '[CLS]': 2,
 'X': 25,
 'U': 26,
 'O': 29}

# Working with the ProtBERT Model for Fine-Tuning:

## Understanding the Task Change

ProtBERT is adept at processing both single sequences and pairs of sequences in the format: `[CLS] Protein Sequence A [SEP] Protein Sequence B [SEP]`.

For our project, we aim to integrate two new tokens that will serve to differentiate between the two proteins in a pair and link various data channels to each other. These tokens will act as markers, identifying different versions of a sequence throughout training, thereby reducing confusion about the relationships between the input channels.

### Approach to Fine-Tuning

This section outlines our approach, which involves selectively training only a few layers of the model to optimize both time and resource utilization.

Initially, the training and modifications will be confined to the last two layers of the model.


In [ ]:
# Freeze layers: only train the top 2 layers
for name, param in model.named_parameters():
    if 'encoder.layer' in name:
        layer_index = int(name.split('.')[3])
        param.requires_grad = layer_index >= 10  # Freeze all but the last 2 layers

#Updated dataset class:
we have added a new parameter to our class called mode so we can tell it to only process the inputs we want.

in this part we only want to work with global input.

in the future we can expand the class and add more properties of the proteins as new input channels

In [ ]:
import torch
from torch.utils.data import Dataset

class ProteinInteractionDataset(Dataset):
    def __init__(self, dataframe, tokenizer, mask_probability=0.15, modes=None):
        """
        Initializes the dataset.

        Args:
            dataframe (pandas.DataFrame): The dataframe containing protein sequences.
            tokenizer (transformers.BertTokenizer): The tokenizer for encoding sequences.
            mask_probability (float): The probability of masking a token for the masked language model.
            modes (list of str): List of modes to prepare data. Options include:
                                 'global_masked' - Returns sequences with random masking.
                                 'local' - Returns non-masked sequences.
                                 Modes can be combined.
        """
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.mask_probability = mask_probability
        self.modes = modes if modes else ['global']  # Default to only global if none specified

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        data = {}

        if 'global' in self.modes:
            global_seq = f"[CLS] {row['Sequence_A']} [ENTITY1] [SEP] {row['Sequence_B']}[ENTITY2] [SEP]"
            input_ids, attention_mask, labels = self.random_mask_sequence(global_seq)
            data['input_ids_global'] = input_ids
            data['attention_mask_global'] = attention_mask
            data['labels_global'] = labels

        if 'local' in self.modes:
            local_seq = f"[CLS] {row['masked_sequence_A']} [ENTITY1] [SEP] {row['masked_sequence_B']}[ENTITY2] [SEP]"
            input_ids, attention_mask = self.tokenize_sequence(local_seq)
            data['input_ids_local'] = input_ids
            data['attention_mask_local'] = attention_mask

        #not applicable now
        if 'coords' in self.modes:
            local_seq = f"[CLS] {row['coords_A']} [ENTITY1] [SEP] {row['coords_B']} [ENTITY2] [SEP]"
            input_ids, attention_mask = self.tokenize_sequence(local_seq)
            data['input_ids_coords'] = input_ids
            data['attention_mask_coords'] = attention_mask

        return data

    def tokenize_sequence(self, sequence):
        encoded = self.tokenizer.encode_plus(
            sequence,
            add_special_tokens=True,
            return_tensors='pt',
            padding=False,
            truncation=True
        )
        return encoded['input_ids'].squeeze(0), encoded['attention_mask'].squeeze(0)

    def random_mask_sequence(self, sequence):
        tokens = self.tokenizer.tokenize(sequence)
        input_ids = torch.tensor(self.tokenizer.convert_tokens_to_ids(tokens), dtype=torch.long)
        labels = torch.full(input_ids.shape, -100)  # Use -100 to ignore these indices in loss calculations
        # Decide where to mask tokens
        mask_indices = torch.rand(input_ids.shape) < self.mask_probability
        labels[mask_indices] = input_ids[mask_indices]
        # 80% of the time, replace masked input tokens with tokenizer.mask_token
        actual_mask = mask_indices & (torch.rand(input_ids.shape) < 0.8)
        input_ids[actual_mask] = self.tokenizer.convert_tokens_to_ids([self.tokenizer.mask_token])[0]
        # 10% of the time, replace masked input tokens with a random token
        random_tokens = torch.randint(2, self.tokenizer.vocab_size, input_ids.shape)
        input_ids[mask_indices & ~actual_mask] = random_tokens[mask_indices & ~actual_mask]
        return input_ids, torch.ones_like(input_ids), labels


In [ ]:
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    batched_data = {}
    for key in ['global', 'local', 'coords']:
        input_ids = [item[f'input_ids_{key}'] for item in batch if f'input_ids_{key}' in item]
        attention_masks = [item[f'attention_mask_{key}'] for item in batch if f'attention_mask_{key}' in item]
        labels = [item.get(f'labels_{key}', None) for item in batch if f'input_ids_{key}' in item]

        if input_ids:
            batched_data[f'input_ids_{key}'] = pad_sequence(input_ids, batch_first=True, padding_value=0)
            batched_data[f'attention_mask_{key}'] = pad_sequence(attention_masks, batch_first=True, padding_value=0)
            if labels:
                # Ensure labels are padded properly and use existing tensors directly where possible
                batched_data[f'labels_{key}'] = pad_sequence([lbl if lbl is not None else torch.full_like(input_ids[0], fill_value=-100) for lbl in labels], batch_first=True, padding_value=-100)

    return batched_data


In [ ]:
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader
from torch.optim import AdamW

# Parameters
batch_size = 32
epochs = 3  # Define the number of epochs to train for

# Assuming 'df' is your DataFrame containing the protein sequences
dataset = ProteinInteractionDataset(pairs_df, tokenizer, mask_probability=0.15, modes=['global'])
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

# Calculate total training steps
total_steps = len(data_loader) * epochs

optimizer = AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=5e-5)

# Setup model, optimizer, and data_loader appropriately
model = model.to(device)  # Make sure model is on the right device
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  # Example optimizer

# Initialize a learning rate scheduler
scheduler = StepLR(optimizer, step_size=1, gamma=0.1)  # Decays the learning rate by a factor of 0.1 every epoch

for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in data_loader:
        input_ids = batch['input_ids_global'].to(device)  # Move tensors to the device
        attention_mask = batch['attention_mask_global'].to(device)
        labels = batch['labels_global'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()  # Update the learning rate

        total_loss += loss.item()

    print(f"Epoch {epoch+1}: Loss {total_loss / len(data_loader):.4f}")

# Save model checkpoint at the end of each epoch
torch.save({
        'epoch': 10,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict(),  # It's a good practice to save the scheduler state as well
        'loss': total_loss / len(data_loader)
    }, f'/content/drive/MyDrive/Phase3Project/checkpoint_epoch_{epoch+1}.pth')




Epoch 1: Loss 15.6344


In [ ]:
'''
#not for here but just remember the loading

import torch
from transformers import BertForMaskedLM, AdamW  # Import the required classes

# Make sure to import or define `tokenizer` and `device` as well
# Assuming `tokenizer` and `device` are defined elsewhere in your code

# Load a checkpoint
checkpoint = torch.load('/content/drive/MyDrive/Checkpoints/checkpoint_epoch_3.pth')

# Reinitialize or define a new model
model = BertForMaskedLM.from_pretrained('Rostlab/prot_bert')
model.resize_token_embeddings(len(tokenizer))
model.load_state_dict(checkpoint['model_state_dict'])
model.to(device)  # Move model to the appropriate device

# Optionally, load the optimizer state if continuing training
optimizer = AdamW(model.parameters(), lr=5e-5)
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

# Model is now ready to be used for inference or to continue training
'''

#Next step: building the Dual channel architecture

##Purpose:
The purpose of this architecture is to enhance the model's ability to learn complex relationships and interactions between proteins or protein regions, which is critical in tasks like protein-protein interaction prediction, understanding functional regions of proteins, and other bioinformatics applications involving sequence data. By using a BERT backbone and a custom attention mechanism, the model can leverage pre-trained language model benefits while adapting to the specific context of protein sequences. The MLM task helps in unsupervised learning, making the model robust in predicting masked token identities based on contextual clues.

In [ ]:
import torch
import torch.nn as nn
from transformers import BertModel, BertTokenizer

class SequenceProcessor(nn.Module):
    def __init__(self, bert_model):
        super(SequenceProcessor, self).__init__()
        self.bert = bert_model

    def forward(self, input_ids, attention_mask=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        return outputs.last_hidden_state  # Get the last hidden states

class CustomAttention(nn.Module):
    def __init__(self, hidden_size):
        super(CustomAttention, self).__init__()
        self.key_layer = nn.Linear(hidden_size, hidden_size)
        self.query_layer = nn.Linear(hidden_size, hidden_size)
        self.value_layer = nn.Linear(hidden_size, hidden_size)
        self.softmax = nn.Softmax(dim=-1)
        self.context_layer = nn.Linear(hidden_size, hidden_size)

    def forward(self, global_features, local_features):
        keys = self.key_layer(global_features)
        queries = self.query_layer(local_features)
        values = self.value_layer(global_features)

        attention_scores = torch.matmul(queries, keys.transpose(-2, -1)) / (keys.size(-1) ** 0.5)
        attention_weights = self.softmax(attention_scores)

        context = torch.matmul(attention_weights, values)
        processed_context = self.context_layer(context)
        return processed_context

class MaskingLayer(nn.Module):
    def __init__(self, tokenizer, mlm_probability=0.15):
        super(MaskingLayer, self).__init__()
        self.tokenizer = tokenizer
        self.mlm_probability = mlm_probability
        self.mask_token_id = tokenizer.mask_token_id

    def forward(self, input_ids):
        # Create a mask to randomly select tokens for masking
        probability_matrix = torch.full(input_ids.shape, self.mlm_probability)
        special_tokens_mask = input_ids.eq(self.tokenizer.pad_token_id) | input_ids.eq(self.tokenizer.cls_token_id) | input_ids.eq(self.tokenizer.sep_token_id)
        probability_matrix.masked_fill_(special_tokens_mask, value=0.0)
        masked_indices = torch.bernoulli(probability_matrix).bool()

        # Only compute loss on masked tokens
        labels = input_ids.clone()
        labels[~masked_indices] = -100  # We do not compute loss for non-masked tokens

        # Replace masked input tokens with tokenizer.mask_token ([MASK])
        input_ids[masked_indices] = self.mask_token_id
        return input_ids, labels

class ProteinInteractionModel(nn.Module):
    def __init__(self, bert_checkpoint_path):
        super(ProteinInteractionModel, self).__init__()
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.tokenizer = BertTokenizer.from_pretrained(bert_checkpoint_path)

        # Load the pretrained BertModel
        bert_model = BertModel.from_pretrained(bert_checkpoint_path)
        bert_model.to(self.device)
        self.sequence_processor = SequenceProcessor(bert_model)

        hidden_size = bert_model.config.hidden_size
        self.masking_layer = MaskingLayer(self.tokenizer).to(self.device)
        self.custom_attention = CustomAttention(hidden_size).to(self.device)
        self.mlm_head = nn.Linear(hidden_size, bert_model.config.vocab_size).to(self.device)

    def forward(self, input_ids_global, attention_mask_global, input_ids_local, attention_mask_local):
        global_features = self.sequence_processor(input_ids_global.to(self.device), attention_mask_global.to(self.device))

        # Apply masking to local input and obtain labels for masked tokens
        masked_input_ids_local, labels_local = self.masking_layer(input_ids_local.to(self.device))
        local_features = self.sequence_processor(masked_input_ids_local, attention_mask_local.to(self.device))

        combined_features = self.custom_attention(global_features, local_features)
        prediction_scores = self.mlm_head(combined_features)

        return prediction_scores, labels_local



#prepering the dataloader, and training set for dual channels

In [ ]:
#the most important part to check if the class definition and data management is correctly working
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

# Splitting the dataset into 80% training and 20% testing
train_df, test_df = train_test_split(pairs_df, test_size=0.2, random_state=42)

# Assuming ProteinInteractionDataset is implemented to handle your DataFrame structure
train_dataset = ProteinInteractionDataset(train_df, tokenizer)
test_dataset = ProteinInteractionDataset(test_df, tokenizer)

#since the model isnt runnint we reduced the batch size to half
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16)


In [ ]:
from transformers import AdamW, get_scheduler
import torch.nn as nn
loss_fn = nn.CrossEntropyLoss()

num_epochs = 5  # Define the number of training epochs
optimizer = AdamW(model.parameters(), lr=5e-5)
scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=len(data_loader)*num_epochs)

model.train()  # Ensure the model is in training mode
for epoch in range(num_epochs):
    total_loss = 0
    for batch in data_loader:
        optimizer.zero_grad()

        input_ids_global = batch['input_ids_global'].to(model.device)
        attention_mask_global = batch['attention_mask_global'].to(model.device)
        labels_global = batch['labels_global'].to(model.device)

        outputs = model(input_ids=input_ids_global, attention_mask=attention_mask_global)
        logits = outputs.logits  # Access logits from the model's output
        loss = loss_fn(logits.view(-1, logits.size(-1)), labels_global.view(-1))
        loss.backward()
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}: Loss {total_loss / len(data_loader):.4f}")


/home/f.ensafitakaldani001/miniconda3/envs/llm_env/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1: Loss 1.7485
Epoch 2: Loss 1.5540
Epoch 3: Loss 1.5443
Epoch 4: Loss 1.5412
Epoch 5: Loss 1.5243


# Prediction!!!!

In [ ]:
sequence_A = "MKTIIALSYIFCLVFADY[MASK]DDK"
sequence_B = "QIKDLLVSSSTDLDTTLVLVNA[MASK]Y"

# Combine into a single sequence, separating pairs with a special token like [SEP]
combined_sequence = f"[ENTITY1] {sequence_A} [SEP] [ENTITY2] {sequence_B} [SEP]"

# Tokenize
encoded_input = tokenizer.encode_plus(combined_sequence, return_tensors="pt")
input_ids = encoded_input['input_ids']
attention_mask = encoded_input['attention_mask']


In [ ]:
# Set device to CUDA if available, else CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move model to the selected device
model.to(device)

# Assuming your input_ids and attention_mask are initially on CPU, move them to the same device
input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)

# Now, you can perform your model inference
model.eval()
with torch.no_grad():
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    predictions = outputs.logits

# Assuming you want to take the argmax to find the most likely token predictions
predicted_token_ids = torch.argmax(predictions, dim=-1)

# If you need to move the predictions back to CPU for further processing or analysis
predicted_token_ids = predicted_token_ids.to('cpu')


In [ ]:
# Convert predicted token IDs to tokens
predicted_tokens = tokenizer.convert_ids_to_tokens(predicted_token_ids[0])

# Locate the masked indices in the original input_ids to print only the predictions for [MASK]
mask_indices = torch.where(input_ids == tokenizer.mask_token_id)[1]

predicted_masked_tokens = [predicted_tokens[idx] for idx in mask_indices]

print("Original Sequence:", combined_sequence)
print("Predicted Tokens for [MASK]:", predicted_masked_tokens)


Original Sequence: [ENTITY1] MKTIIALSYIFCLVFADY[MASK]DDK [SEP] [ENTITY2] QIKDLLVSSSTDLDTTLVLVNA[MASK]Y [SEP]
Predicted Tokens for [MASK]: ['[UNK]', '[UNK]']
